## MV Imputation

### 3 Alternativas (atleast 2):
1. Eliminate Columns & Rows bove certain Threshold
- **1.1** Eliminate Columns with percentage(%) of Missing Values above 15% 'Credit_Mix' && 'Monthly_Inhand_Salary'
- **1.2** Eliminate Rows with percentage(%) of Missing Values above 16%
- **1.3** Fill Missing Values using Mean for Numerics and Most-frequent to Symbolic

2. Filling Missing Values (median & most_frequent)

3. Don´t eliminate any Columns or Rows (knnimputer)

In [3]:
import pandas as pd
from matplotlib.pyplot import figure, show
from sklearn.impute import SimpleImputer
import dslabs_functions as dslabs
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import savefig
from sklearn.impute import KNNImputer

filename = '../data_preparation_csvs/enconding_csvs/data_encoded.csv'
data_encoded: pd.DataFrame = pd.read_csv(filename, index_col='ID', sep=',', decimal='.', na_values='')
print(f"Dataset nr records={data_encoded.shape[0]}", f"nr variables={data_encoded.shape[1]}")
##print(data_encoded.dtypes)

most_frequent = ['Month', 'Occupation', 'Payment_of_Min_Amount', 'Credit_Mix', 'Monthly_Inhand_Salary'
                 , 'Payment_Behaviour', 'Payday Loan', 'Personal Loan', 'Debt Consolidation Loan'
                 , 'Auto Loan', 'Not Specified Loan', 'Student Loan', 'Credit-Builder Loan'
                 , 'Mortgage Loan', 'Home Equity Loan', 'Credit_Score']

## Strategies to be used
imputer_mean = SimpleImputer(strategy='mean')
imputer_mode = SimpleImputer(strategy='most_frequent')
imputer_median = SimpleImputer(strategy='median')
imputer_knn = KNNImputer(n_neighbors=5)

Dataset nr records=100000 nr variables=32


In [2]:
def save(df, name):
    df.to_csv(f'../data_preparation_csvs/mv_imputation_csvs/imputed_{name}.csv', index=True)

    ## Separate into Train and Test and save in different files
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

    ## Save the training and testing sets to CSV files
    train_data.to_csv('../data_preparation_csvs/mv_imputation_csvs/train_data_mv_{name}.csv', index=True)
    test_data.to_csv('../data_preparation_csvs/mv_imputation_csvs/test_data_mv_{name}.csv', index=True)

    ## Evaluate Approach
    file_tag = "Credit_Score"
    target = "Credit_Score"

    figure()
    eval: dict[str, list] = dslabs.evaluate_approach(train_data, test_data, target=target, metric="recall")
    dslabs.plot_multibar_chart(
        ["NB", "KNN"], eval, title=f"{file_tag} evaluation", percentage=True
    )
    savefig(f"../data_preparation_images/missing_values_imputation_result/data_mv_{name}_eval.png")
    show()

In [5]:
# Cross-validation analysis to determine best k

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

new_data = data_encoded.dropna()

n_neighbors = 100000
n_folds = 5
kf = KFold(n_splits=n_folds)

accuracies = []

for train_index, val_index in kf.split(new_data):
    X_train, X_val = new_data.iloc[train_index], new_data.iloc[val_index]
    y_train, y_val = new_data["Credit_Score"].iloc[train_index], new_data["Credit_Score"].iloc[val_index]
    
    # Initialize kNN classifier
    knn = KNeighborsClassifier(n_neighbors=int(100))
    
    # Train the model
    knn.fit(X_train, y_train)
    
    # Make predictions
    predictions = knn.predict(X_val)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_val, predictions)
    accuracies.append(accuracy)

average_accuracy = sum(accuracies) / len(accuracies)
print("Average Accuracy:", average_accuracy)

Average Accuracy: 0.7623191182098081


In [1]:
# Alternative 3

knn_imputed = pd.DataFrame(imputer_knn.fit_transform(data_encoded))

save(knn_imputed, "alt3")

NameError: name 'pd' is not defined